<a href="https://colab.research.google.com/github/pachterlab/GRNP_2020/blob/master/notebooks/figure_generation/GenFigS6-S19Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Precalculates data for the supplementary figures S6-S19**

This notebook precalculates the data for the supplementary figures S6-S19, since there are some heavy calculation steps involved for generating the figures. The most demanding task is prediction of unseen molecules for each gene using the ZTNB method. This notebook may take several hours to run.

Steps:
1. Download the code and processed data
2. Setup the R environment
3. Define a general function to precalculate figure data for a dataset and save it to disk
4. Call the precalculation function for all datasets

**1. Download the code and processed data**

In [1]:
#download the R code
![ -d "GRNP_2020" ] && rm -r GRNP_2020

!git clone https://github.com/pachterlab/GRNP_2020.git


Cloning into 'GRNP_2020'...
remote: Enumerating objects: 442, done.
remote: Counting objects: 100% (442/442), done.
remote: Compressing objects: 100% (362/362), done.
remote: Total 865 (delta 287), reused 142 (delta 80), pack-reused 423
Receiving objects: 100% (865/865), 7.46 MiB | 13.26 MiB/s, done.
Resolving deltas: 100% (515/515), done.


In [2]:
#download processed data from Zenodo for all datasets
![ -d "data" ] && rm -r data
!mkdir figureData
!cd figureData && wget https://zenodo.org/record/3909758/files/EVAL.zip?download=1 && unzip 'EVAL.zip?download=1' && rm 'EVAL.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/EVALPBMC.zip?download=1 && unzip 'EVALPBMC.zip?download=1' && rm 'EVALPBMC.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/EVALPBMC_DS.zip?download=1 && unzip 'EVALPBMC_DS.zip?download=1' && rm 'EVALPBMC_DS.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/EVALPBMC_SW.zip?download=1 && unzip 'EVALPBMC_SW.zip?download=1' && rm 'EVALPBMC_SW.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/MRET.zip?download=1 && unzip 'MRET.zip?download=1' && rm 'MRET.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/MRET2.zip?download=1 && unzip 'MRET2.zip?download=1' && rm 'MRET2.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/LC.zip?download=1 && unzip 'LC.zip?download=1' && rm 'LC.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_V2.zip?download=1 && unzip 'PBMC_V2.zip?download=1' && rm 'PBMC_V2.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_V3.zip?download=1 && unzip 'PBMC_V3.zip?download=1' && rm 'PBMC_V3.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_V3_2.zip?download=1 && unzip 'PBMC_V3_2.zip?download=1' && rm 'PBMC_V3_2.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_V3_3.zip?download=1 && unzip 'PBMC_V3_3.zip?download=1' && rm 'PBMC_V3_3.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_NG.zip?download=1 && unzip 'PBMC_NG.zip?download=1' && rm 'PBMC_NG.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_NG_2.zip?download=1 && unzip 'PBMC_NG_2.zip?download=1' && rm 'PBMC_NG_2.zip?download=1'


--2020-07-02 12:05:17--  https://zenodo.org/record/3909758/files/EVAL.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206479312 (197M) [application/octet-stream]
Saving to: ‘EVAL.zip?download=1’

EVAL.zip?download=1 100%[===================>] 196.91M  26.1MB/s    in 8.3s    

2020-07-02 12:05:26 (23.8 MB/s) - ‘EVAL.zip?download=1’ saved [206479312/206479312]

Archive:  EVAL.zip?download=1
   creating: EVAL/
  inflating: EVAL/Bug_10.RData       
  inflating: EVAL/Bug_100.RData      
  inflating: EVAL/Bug_20.RData       
  inflating: EVAL/Bug_25.RData       
  inflating: EVAL/Bug_40.RData       
  inflating: EVAL/Bug_5.RData        
  inflating: EVAL/Bug_60.RData       
  inflating: EVAL/Bug_80.RData       
  inflating: EVAL/ds_summary.txt     
  inflating: EVAL/PredEvalData.RDS   
  inflating: EVAL/Stats.RData        
--2020-07-02 12:05:29--

In [3]:
#Check that download worked
!cd data && ls -l && cd EVAL && ls -l

/bin/bash: line 0: cd: data: No such file or directory


**2. Prepare the R environment**

In [4]:
#switch to R mode
%reload_ext rpy2.ipython


/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [5]:
#install the R packages and setup paths
%%R
#install.packages("qdapTools")
install.packages("dplyr")
install.packages("preseqR")
#install.packages("stringdist")


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/dplyr_1.0.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 836651 bytes (817 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write 

**3. Define a general function to precalculate data for a dataset**

This function calculates all prediction data needed for the figures. For the A figures, all UMIs from all genes are joined into a single pool, and prediction is done from there. For the other figures (B-F), the predictions are done per gene. We predict using four methods:

1. Preseq DS (Rational functions approximation), trunkating CU histograms at 2
2. Preseq DS, trunkating CU histograms at 20
3. Zero-trunkated negative binomial (ZTNB)
4. "Best practice", which selects Preseq DS (here with histograms trunkated at 2) if the number of copies per molecule CV > 1, otherwise ZTNB.

In [6]:
#create output directory
!mkdir figureData

mkdir: cannot create directory ‘figureData’: File exists


In [7]:
#First set some path variables
%%R
source("GRNP_2020/RCode/pathsGoogleColab.R")


In [8]:
#Import the code for prediction (available in other notebooks)
%%R
source(paste0(sourcePath,"ButterflyHelpers.R"))
source(paste0(sourcePath,"preseqHelpers.R"))





R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [9]:
#Define a function that precalculates the figure data for a dataset
%%R
GenAlgEvaluationData <- function(dsid) {
  print(paste0("Processing ",dsid, ":"))
  loadStats(dsid)
  stats = getStats(dsid)
  
  
  ##################################
  #first, prediction of all UMIs
  ##################################
  
  #Get UMI counts at all different ds stages
  ###########################################
  
  dss = c("5","10","20","40","60","80","100")
  
  #generate the columns to extract from stats
  colList = dss #allocation of right size
  for (i in 1:length(dss)) {
    colList[i] = paste0("UMIs_", dsid, "_d_", dss[i])
  }
  extrFromStats = stats[,which(colnames(stats) %in% colList)]
  dsCounts = as.numeric(colSums(extrFromStats))

  
  #get histogram from 0.05:
  loadBug(dsid, 0.05)
  h = totalCPUHistogram(getBug(dsid, 0.05))
  rmBug(dsid, 0.05)

  x = c(0.05,0.1,0.2,0.4,0.6,0.8,1)
  t = c(1,2,4,8,12,16,20)


  #predict with Good-Toulmin
  predGT005 = rep(0,length(t))
  predGT005[1] = dsCounts[1];
  for (i in 2:length(t)) {
    predGT005[i] = goodToulmin(h,t[i])
  }
  
  #predict with Preseq DS, mt=20
  predPSDS005_20 = rep(0,length(t))
  predPSDS005_20[1] = dsCounts[1];
  for (i in 2:length(t)) {
    predPSDS005_20[i] = predPreSeqDS(h,t[i],20)
  }
  
  #predict with Preseq DS, mt=2
  predPSDS005_2 = rep(0,length(t))
  predPSDS005_2[1] = dsCounts[1];
  for (i in 2:length(t)) {
    predPSDS005_2[i] = predPreSeqDS(h,t[i],2)
  }
  
  #predict with Preseq ZTNB
  predPSZTNB005 = rep(0,length(t))
  predPSZTNB005[1] = dsCounts[1];
  for (i in 2:length(t)) {
    predPSZTNB005[i] = predPreSeqZTNB(h,t[i]) #just ignore the warnings, deprecated...
  }
  
  ##################################
  #now, prediction per gene for different methods
  ##################################
  
  loadBug(dsid, 0.1)
  ds10Bug = getBug(dsid, 0.1)
  rmBug(dsid, 0.1)

  collapsed10 = aggregate(count~gene, ds10Bug, FUN=c) #if you get an error here, you probably defined a variable called "c"...
  totUMIs10 = sapply(collapsed10$count, FUN=length)
  rm(ds10Bug)
  
  loadBug(dsid, 1)
  ds100Bug = getBug(dsid, 1)
  rmBug(dsid, 1)
  
  collapsedFull = aggregate(count~gene, ds100Bug, FUN=c) #if you get an error here, you probably defined a variable called "c"...
  rm(ds100Bug)
  merged2 = inner_join(collapsed10, collapsedFull, by="gene")
  rm(collapsed10, collapsedFull)
  colnames(merged2) = c("gene", "DS10", "Full")
  
  totUMIsFull = sapply(merged2$Full, FUN=length)
  #sort the genes on number of UMIs
  srt = sort(totUMIs10, index.return=T)
  umis = srt$x
  merged2srt = merged2[srt$ix,]
  
  #now predict using both ztnb and ds:
  numgenes = dim(merged2srt)[1]
  predds_20 = rep(0,numgenes)
  predds_2 = rep(0,numgenes)
  predztnb = rep(0,numgenes)
  predbp = rep(0,numgenes)
  predscaled = rep(0,numgenes)
  fracOnes = rep(0,numgenes)
  fullUMIs = rep(0,numgenes)
  
  globScale = sum(totUMIsFull)/sum(totUMIs10)

  #All quick ones.
  print(paste0("DS etc.:", numgenes))
  for (i in 1:numgenes) {
    if (i %% 1000 == 0) {
      print(i)
    }
    h = hist(merged2srt$DS10[[i]], breaks=seq(0.5, max(merged2srt$DS10[[i]])+0.5, by=1), plot = F)
    predds_20[[i]] = predPreSeqDS(h, 10, 20)
    predds_2[[i]] = predPreSeqDS(h, 10, 2)
    predscaled[[i]] = length(merged2srt$DS10[[i]])*globScale #this resembles CPM
    
    fullUMIs[[i]] = length(merged2srt$Full[[i]])
    
    fracOnes[[i]] = h$density[1]
  }
  
  #ztnb
  print(paste0("ZTNB:", numgenes))
  for (i in 1:numgenes) {
    if (i %% 1000 == 0) {
      print(i)
    }
    h = hist(merged2srt$DS10[[i]], breaks=seq(0.5, max(merged2srt$DS10[[i]])+0.5, by=1), plot = F)
    predztnb[[i]] = predPreSeqZTNB(h, 10)
  }
  
  #best practice
  print(paste0("Best practice:", numgenes))
  for (i in 1:numgenes) {
    if (i %% 1000 == 0) {
      print(i)
    }
    h = hist(merged2srt$DS10[[i]], breaks=seq(0.5, max(merged2srt$DS10[[i]])+0.5, by=1), plot = F)
    predbp[[i]] = predPreSeq(h, 10, mt=2)
  }
  
  
  toSave = list(x, dsCounts, predGT005, predPSDS005_2, predPSDS005_20, predPSZTNB005, 
                predds_20, predds_2, predztnb, predbp, predscaled, fracOnes, fullUMIs, umis,
                merged2srt)
  
  
  filename = paste0(figure_data_path, dsid, "/PredEvalData.RDS")
  saveRDS(toSave, filename)
  
  
}


**4. Call the precalculation function for all datasets**


In [ ]:
%%R
GenAlgEvaluationData("EVAL")
GenAlgEvaluationData("EVALPBMC")
GenAlgEvaluationData("EVALPBMC_DS")
GenAlgEvaluationData("EVALPBMC_SW")
GenAlgEvaluationData("PBMC_V3")
GenAlgEvaluationData("PBMC_V3_2")
GenAlgEvaluationData("PBMC_V3_3")
GenAlgEvaluationData("PBMC_NG")
GenAlgEvaluationData("PBMC_NG_2")
GenAlgEvaluationData("PBMC_V2")
GenAlgEvaluationData("LC")
GenAlgEvaluationData("MRET")
GenAlgEvaluationData("MRET2")


[1] "Processing EVAL:"
[1] "DS etc.:17546"
[1] 1000
[1] 2000
[1] 3000
[1] 4000
[1] 5000
[1] 6000
[1] 7000
[1] 8000
[1] 9000
[1] 10000
[1] 11000
[1] 12000
[1] 13000
[1] 14000
[1] 15000
[1] 16000
[1] 17000
[1] "ZTNB:17546"
[1] 1000
[1] 2000
[1] 3000


In [ ]:
!cd figureData && ls -l && cd EVAL && ls -l